In [74]:
import glob
import pathlib

import pandas as pd
pd.get_option("display.max_columns")
import plotly.express as px

from btree_analysis.log_parser import parse_experiment

In [17]:
EXPERIMENT_PATH = "/Users/artur/Documents/broadcast-tree/experiments/results/*"
EXPERIMENT_IDENTIFIER = ["payload_size", "flood", "unchanged_counter", "source_id", "iteration"]

In [18]:
events = []
for experiment in glob.glob(EXPERIMENT_PATH):
    _events, _, _ = parse_experiment(pathlib.Path(experiment))
    events += _events
    
df = pd.DataFrame(events)

Node 77 switched parents 0 times
Node 1 switched parents 0 times
Node 51 switched parents 0 times
Node 8 switched parents 0 times
Node 60 switched parents 0 times
Node 44 switched parents 0 times
Node 37 switched parents 0 times
Node 23 switched parents 0 times
Node 42 switched parents 0 times
Node 50 switched parents 0 times
Node 75 switched parents 0 times
Node 58 switched parents 0 times
Node 32 switched parents 0 times
Node 26 switched parents 0 times
Node 38 switched parents 0 times
Node 45 switched parents 0 times
Node 18 switched parents 0 times
Node 79 switched parents 0 times
Node 0 switched parents 0 times
Node 68 switched parents 0 times
Node 28 switched parents 0 times
Node 72 switched parents 0 times
Node 40 switched parents 0 times
Node 41 switched parents 0 times
Node 54 switched parents 0 times
Node 83 switched parents 0 times
Node 25 switched parents 0 times
Node 49 switched parents 0 times
Node 57 switched parents 0 times
Node 76 switched parents 0 times
Node 24 switc

In [75]:
df[df["event"] == "error"]

,event,timestamp,node_id,payload_size,flood,poll_timeout,discovery_bcast_interval,pending_timeout,source_retransmit_payload,unchanged_counter,...,iteration,source_id,message,tx_pwr,data_len,level,parent,mW,mWb,frame_type
1338,error,2022-07-18 16:44:02.580862,32,100K,True,100,100,1000,100,5,...,0,0,Could not set txpower.,NaN,NaN,ERROR,NaN,NaN,NaN,NaN
8694,error,2022-07-18 16:44:09.063603,59,100K,True,100,100,1000,100,5,...,0,0,Could not set txpower.,NaN,NaN,ERROR,NaN,NaN,NaN,NaN
8698,error,2022-07-18 16:44:09.064753,59,100K,True,100,100,1000,100,5,...,0,0,Retrying to set tx power.,NaN,NaN,WARN,NaN,NaN,NaN,NaN
11163,error,2022-07-18 16:44:11.466358,48,100K,True,100,100,1000,100,5,...,0,0,Could not set txpower.,NaN,NaN,ERROR,NaN,NaN,NaN,NaN
11166,error,2022-07-18 16:44:11.467484,48,100K,True,100,100,1000,100,5,...,0,0,Retrying to set tx power.,NaN,NaN,WARN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2255102,error,2022-07-18 16:07:38.141345,45,1K,False,100,100,1000,100,25,...,0,54,Retrying to set tx power.,NaN,NaN,WARN,NaN,NaN,NaN,NaN
2256045,error,2022-07-18 16:07:38.929434,42,1K,False,100,100,1000,100,25,...,0,54,Pending parent did not respond in time. Removi...,NaN,NaN,WARN,NaN,NaN,NaN,NaN
2256845,error,2022-07-18 16:07:39.665187,33,1K,False,100,100,1000,100,25,...,0,54,Could not set txpower.,NaN,NaN,ERROR,NaN,NaN,NaN,NaN
2256847,error,2022-07-18 16:07:39.665306,33,1K,False,100,100,1000,100,25,...,0,54,Retrying to set tx power.,NaN,NaN,WARN,NaN,NaN,NaN,NaN


# Evaluation Metrics
- [x] Table: How many nodes have received the entire payload?
- [x] Plot: Percentage comparison of BTP vs. flood w.r.t. to received payloads
- [X] Plot: How many percent more or less energy does BTP need compared to flood
- [X] Plot: What is the overhead of the BTP protocol (i.e. payload energy vs. protocol energy)?
- [X] Plot: How are individual nodes strained?

# Absolute Values of Received Payloads

In [28]:
reception_df = df.groupby(EXPERIMENT_IDENTIFIER)["event"].apply(lambda x: (x == 'receive').sum()).reset_index(name='count')

In [29]:
reception_df

,payload_size,flood,unchanged_counter,source_id,iteration,count
0,100K,False,5,0,0,8
1,100K,False,5,54,0,3
2,100K,False,5,80,0,14
3,100K,False,15,0,0,5
4,100K,False,15,54,0,7
5,100K,False,15,80,0,18
6,100K,False,25,0,0,5
7,100K,False,25,54,0,2
8,100K,False,25,80,0,13
9,100K,True,5,0,0,12


In [68]:
reception_df[reception_df["count"] == 0]

,payload_size,flood,unchanged_counter,source_id,iteration,count
14,100K,True,15,80,0,0
15,100K,True,25,0,0,0
16,100K,True,25,54,0,0
17,100K,True,25,80,0,0
27,1K,True,5,0,0,0
28,1K,True,5,54,0,0
29,1K,True,5,80,0,0
45,1M,True,5,0,0,0
46,1M,True,5,54,0,0
47,1M,True,5,80,0,0


In [31]:
fig = px.box(
    reception_df,
    x="payload_size",
    y="count", 
    color="unchanged_counter",
    facet_col="flood",
    facet_row="source_id"
)
fig

# Relative Values of Received Payloads

In [36]:
def reception_ratio(x):
    flooding = x[x["flood"] == True]
    btp = x[x["flood"] == False]
    
    return float(btp["count"]) / flooding["count"]

count_df = reception_df.groupby(["payload_size", "source_id", "unchanged_counter", "iteration"]).apply(reception_ratio).reset_index(name='ratio')

In [39]:
fig = px.box(
    count_df,
    x="payload_size",
    y="ratio", 
    color="unchanged_counter",
    facet_col="source_id"
)
fig

# Prepare energy df

In [52]:
def energy_ratio(x):
    energy_payload_df = x[x["frame_type"] == "payload"]
    energy_protocol_df = x[x["frame_type"] == "protocol"]
    
    energy_payload = float(energy_payload_df["mWb_total"])
    
    if energy_protocol_df.empty:
        energy_protocol = 0
    else:
        energy_protocol = float(energy_protocol_df["mWb_total"])
    
    total_energy = energy_payload + energy_protocol
    
    payload_rel = energy_payload / total_energy
    protocol_rel = energy_protocol / total_energy
    
    x.loc[x["frame_type"] == "payload", "mWb_rel"] = payload_rel
    x.loc[x["frame_type"] == "protocol", "mWb_rel"] = protocol_rel
    
    return x
    
    

df["mW"] = 10**(df["tx_pwr"]/10)
df["mWb"] = df["mW"] * (df["data_len"] + 34)

df.loc[(df["event"] == "send") & (df["message"] == "Successfully forwarded payload.") | (df["message"] == "Successfully sent next chunk."), "frame_type"] = "payload"
df.loc[(df["event"] == "send") & ((df["message"] != "Successfully forwarded payload.") & (df["message"] != "Successfully sent next chunk.")), "frame_type"] = "protocol"

energy_df = df[df["event"] == "send"].groupby(EXPERIMENT_IDENTIFIER + ["frame_type"])["mWb"].sum().reset_index(name='mWb_total')
energy_df = energy_df.groupby(EXPERIMENT_IDENTIFIER).apply(energy_ratio)#.reset_index()


# Comparison between flooding and BTP, energy wise

In [61]:
def energy_ratio(x):
    flooding = x[(x["flood"] == True)].sum()
    btp = x[(x["flood"] == False)].sum()
    
    return btp["mWb_total"] / flooding["mWb_total"]

energy_ratio_df = energy_df.groupby(["payload_size", "source_id", "unchanged_counter", "iteration"]).apply(energy_ratio).reset_index(name='ratio')

/var/folders/26/2pp7vp553ms3s26tctkklxzr0000gp/T/ipykernel_58304/242195146.py:5: RuntimeWarning:

divide by zero encountered in double_scalars



In [62]:
fig = px.box(
    energy_ratio_df,
    x="payload_size",
    y="ratio", 
    color="unchanged_counter",
    facet_col="source_id"
)
fig

# What is BTP's overhead for building the tree

In [63]:
btp_energy_df = energy_df[energy_df["flood"] == False]

fig = px.bar(
    btp_energy_df,
    x="payload_size",
    y="mWb_rel",
    color="frame_type",
    facet_col="unchanged_counter",
    facet_row="source_id"
)
fig

# Strain on Individual Nodes

In [66]:
def sort_groups(x):
    sorted_x = x.sort_values(by="amount", ascending=False)
    sorted_x['sorted'] = range(len(sorted_x))
    
    return sorted_x

# strain_df = df.drop(df[df["flood"] == True].index)

strain_df = df.groupby(EXPERIMENT_IDENTIFIER + ["node_id"])["mWb"].sum().reset_index(name="amount")

strain_df = strain_df.groupby(EXPERIMENT_IDENTIFIER).agg(sort_groups).reset_index(drop=True)

strain_df["normalized"] = (strain_df["amount"] - strain_df["amount"].min()) / (strain_df["amount"].max() - strain_df["amount"].min())

/var/folders/26/2pp7vp553ms3s26tctkklxzr0000gp/T/ipykernel_58304/1837664311.py:11: FutureWarning:

Dropping invalid columns in DataFrameGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the function.



In [67]:
fig = px.line(
    strain_df,
    x="sorted",
    y="normalized",
    color="flood",
    facet_col="unchanged_counter",
    facet_row="source_id",
    line_dash="payload_size"
)
fig